In [1]:
import lzma
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.linear_model import LinearRegression

In [2]:
#solve linear regression
def linear_regression(x, y):
    X = x.reshape(-1, 1) 
    reg = LinearRegression(fit_intercept=True, normalize=True)
    f = reg.fit(X, y)
    a = reg.coef_[0]
    b = reg.intercept_
    return a, b

In [3]:
#function to read TSV file into pandas DataFrame
def read_xz(filename):
    #read TSV file into pandas DataFrame
    data_lines = []
    with lzma.open(filename, mode='rt',encoding='Latin1') as file:
    # with lzma.open('new_york/new_york_subsampled_metadata.tsv.xz', mode='rt',encoding='Latin1') as file:
        for line in file:
            data_lines.append(line)

    data_lines = [re.split('\t', l) for l in data_lines]
    header=data_lines[0]
    data_lines.pop(0)
    df = pd.DataFrame(data_lines)
    df.columns = header
    return df


In [4]:
df = read_xz('../ny_results/combined_metadata.tsv.xz')
b11 = df[df["pango_lineage"] == "B.1.1.7"]
b15 = df[df["pango_lineage"] == "B.1.526"]

In [6]:
#filter for different regions and variants
u11 = b11[b11["region"] == "North America"]
u15 = b15[b15["region"] == "North America"]
ny11 = b11[b11["ny"] == "yes"] #no rows?
ny15 = b15[b15["ny"] == "yes"]

def get_frequency(mdf):
    #get counts by day
    mdf.sort_values(by=['date'])
    r_df = mdf.groupby(['date']).count()
    r_df = r_df.iloc[:,:1]
    r_df.reset_index(level=0, drop=True)
    #get rolling 7 day average
    r_df=r_df.rolling(window=7).mean() 
    r_df=r_df.dropna()
    #rename columns (now we have counts)
    r_df.rename(columns = {'strain':'count'}, inplace = True)
    r_sr = r_df['count']
    vals = r_sr.values
    return r_sr,vals

#setup plots
titles = ["B.1.1.7 Growth","B.1.526 Growth","B.1.526 NY Growth","B.1.1.7"]
fig, ([axs1,axs2],[axs3,axs4])  = plt.subplots(2, 2,figsize=(20, 12))
axs= fig.get_axes()
to_plot = [u11,u15,ny15]
for i in range(0,len(to_plot)):
    r_sr,vals = get_frequency(to_plot[i])
    #convert dates to numbers for regression
    dts = np.array(pd.to_datetime(r_sr.index.values).astype(np.int64))
    #linear regression (logistic regression model is not giving good results)
    a,b = linear_regression(dts, vals)

    #setup plot
    y_hat = a * dts + b
    spacing = 0.200
    #ax = fig.add_subplot(1, 1, 1)
    axs[i].plot(pd.to_datetime(dts), vals, 'o',label='Frequency')
    axs[i].plot(pd.to_datetime(dts), y_hat, '-', label='Linear Regression')
    axs[i].set_ylabel('7 day average')
    axs[i].set_title(titles[i])
    axs[i].autoscale()
    axs[i].legend(loc='upper right')
    fig.subplots_adjust(bottom=spacing)
    axs[i].set_ylabel('7 day ave.')


,strain,virus,gisaid_epi_isl,genbank_accession,date,region,country,division,location,region_exposure,...,authors,url,title,paper_url,date_submitted,purpose_of_sequencing,variant,gisaid,ny,reference\n
